In [1]:
from src.data.dataset import CelebADataset
from src.data.preprocessing import (
    pre_transform,
    augmentation_transforms,
    post_transform
)

from src.models.efficientnet import EfficientNetModel
from src.pipeline.train_model import train
from src.pipeline.save_load_model import (
    save_model,
    load_model
)

from src.models.inference_model import FaceRec

import albumentations as albu
from torch import nn, optim
import torch

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


EMBEDDING_SIZE = 300
MARGIN = 1.0

N_ANCHOR_CLASSES = 10

BATCH_SIZE = 64
MOMENTUM = 0.9
LR = 1e-3

In [3]:
basic_transformation = albu.Compose([
    pre_transform(), post_transform()
])

augmentation_transformation = albu.Compose([
    pre_transform(), augmentation_transforms(), post_transform()
])

dataset = CelebADataset(
    basic_transforms=basic_transformation,
    augmentation_transforms=augmentation_transformation
)

Reading images:   0%|          | 0/202599 [00:00<?, ?it/s]

In [4]:
model = EfficientNetModel(EMBEDDING_SIZE).to(DEVICE)
model.train()
opt = optim.SGD(model.get_trainable_params(), lr=LR, momentum=MOMENTUM)
scheduler = optim.lr_scheduler.StepLR(opt, 1, gamma=0.8, last_epoch=-1)

triplet_loss = nn.TripletMarginLoss(margin=MARGIN, p=2.0)

In [5]:
history = train(
    model=model,
    train_dataset=dataset,
    loss_func=triplet_loss,
    opt=opt,
    scheduler=scheduler,
    session_size=2,
    epoch_num=5,
    tensorboard=True
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
save_model(
    model=model,
    model_name="efficient_1"
)